# ROC curve exercise

In [ ]:
from sklearn import datasets
import numpy as np
from matplotlib import pyplot as plt
from matplotlib.colors import ListedColormap

from ipywidgets import widgets,interactive
from sklearn.metrics import roc_curve

This section loads the data and plots it

In [ ]:
cancer=datasets.load_breast_cancer()
#only take two features
X=cancer.data[:,:2]
y=cancer.target
malignent=cancer.data[y==1]
benign=cancer.data[y==0]
xb,yb=benign[:,0],benign[:,1]
xm,ym=malignent[:,0],malignent[:,1]
plt.scatter(xb,yb,color='g',alpha=0.2,label='benign')
plt.scatter(xm,ym,color='r',alpha=0.2,label='malignent')
plt.legend();

## Task 1

Now we use `sklearn` to train a logistic regression model to classify our data. We will discuss the meaning of `C` in a later lecture. 

In [ ]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression(C=1000.0, random_state=0, solver='lbfgs')
lr.fit(X,y)

Use the `decision_function` member function of the linear regression model to create four arrays containing the z values correspoinding to true positive, false positive, etc. in the function `getDecisionArrays`. The return values of this function should be four one-dimensional arrays of floats. 

In [ ]:
def getDecisionArrays(X, y, model):
    ''' Return tp, fp, tn, fn, four arrays of values of z corresponding to true positive, 
        false positive, and false negative.
    '''
    tp = []
    tn = []
    fp = []
    fn = []
    
    res_lr = model.decision_function(X)
    for ii in range(len(res_lr)):
        if res_lr[ii]>0:
            if y[ii]==1:
                tp.append(res_lr[ii])
            else:
                fp.append(res_lr[ii])
        else:
            if y[ii]==1:
                fn.append(res_lr[ii])
            else:
                tn.append(res_lr[ii])
    return tp, fp, tn, fn


In [ ]:
tp, fp, tn, fn = getDecisionArrays(X, y, lr)
assert len(tp) == 334, 'There should be 334 events which are correctly classified as positive.'
assert np.isclose(tp[:5],np.array([2.403741551788574, 2.5955164324497595, 7.08899451935018, 2.0572141926784724, 7.511859816549476])).all()
assert len(fp) == 39, 'There should be 39 events which are incorrectly classified as positive.'
assert np.isclose(fp[:5],np.array([3.33158079625165, 3.2636661961861755, 0.8126760344878186, 1.3472541433135312, 1.433815447821221])).all()
assert len(tn) == 173
assert np.isclose(tn[:3],np.array([-1.4321272777718903, -5.771474037175768, -5.600356454622837])).all()
assert len(fn) == 23
assert np.isclose(fn[:3],np.array([-0.845141907046763, -0.048752379756617614, -2.19717345542551])).all()

Use the `getDecisionArrays` function, which you defined above, to reproduce the following plot

![](yofz_colors.png)

The definitions of $y$ and $z$ are the same as in the lecture notes.

In [ ]:
tp, fp, tn, fn = getDecisionArrays(X, y, lr)
'''Create your plot here'''
plt.scatter(tp,np.ones(len(tp)),  label = 'TP', alpha=0.3)
plt.scatter(fp,np.zeros(len(fp)), label = 'FP', alpha=0.3)
plt.scatter(tn,np.zeros(len(tn)), label = 'TN', alpha=0.3)
plt.scatter(fn,np.ones(len(fn)),  label = 'FN', alpha=0.3)
plt.xlabel('z')
plt.ylabel('y')
plt.axvline(linewidth=2, color='black')
plt.legend()
plt.show()

## Task 2

Create a ROC curve for this logistic regression model.

In [ ]:
'''Plot your ROC curve here'''
res_lr = lr.decision_function(X)
res_matrix = np.column_stack((res_lr,y))
res_matrix = res_matrix[res_matrix[:,0].argsort()]
true_val = np.count_nonzero(y == 1)
false_val = len(y)-true_val
count_fn = 0
count_tn = 0
tpr = []
fpr = []
for ii in range(len(res_matrix)):
    if(res_matrix[ii,1]==0):
        count_tn += 1
    else:
        count_fn += 1
    count_tp = true_val - count_fn
    count_fp = false_val - count_tn
    tpr.append(count_tp/true_val)
    fpr.append(count_fp/false_val)
    
plt.plot(fpr,tpr)
plt.xlabel('FPR')
plt.ylabel('TPR')
plt.plot()

## Bonus exercise 1


Now that we did the curve the hard way we can check that we got the right answer using `sklearn` and the `roc_curve` member of the model. Look up the description of `roc_curve` and use it to make another plot of the ROC curve.

In [ ]:
'''Plot your ROC curve using the roc_curve function of sklear here'''

fpr,tpr, threshold = roc_curve(y,res_lr)
plt.plot(fpr,tpr)
plt.xlabel('FPR')
plt.ylabel('TPR')
plt.show()

## Bonus exercise 2 

Create an interactive plot showing the location on the ROC curve and the plot you coded above with the location of the threshold as an interactive parameter.   

In [ ]:

def ROCinteractive(t):
    f, (ax1, ax2) = plt.subplots(1,2,figsize=(16,8))
    # replace this...
    vs = np.linspace(0,1)
    ax1.plot(vs,vs)

    true_positive_rate,false_positive_rate = np.cos(t)**2, np.sin(t)**2 
    ax1.plot([false_positive_rate],[true_positive_rate],'or')
    ax1.set_xlabel('false positive rate')
    ax1.set_ylabel('true positive rate')

    ax2.axvline(t,color='r')
    ax2.set_xlim((-15,10))
    ax2.set_ylim((-0.1,1.1))

st = widgets.FloatSlider(min = -15, max = 10, value = 0.,continuous_update=False)
i=interactive(ROCinteractive, t = st)
i.layout.height='700px'
display(i);
